Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "SAI LAKSHMI NAVYA MADDU"
COLLABORATORS = "PROF.ZIA"

---

In [2]:
from IPython.display import display, HTML
import pandas as pd
import sqlite3
from sqlite3 import Error

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql):
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows


In [3]:
conn = create_connection('non_normalized.db')
sql_statement = "select * from Students;"
df = pd.read_sql_query(sql_statement, conn)
display(df)

,StudentID,Name,Degree,Exams,Scores
0,1,"Rodriguez, Pamela",graduate,"exam7 (2017), exam9 (2018), exam3 (2018), exam...","61, 38, 85, 70, 44, 43, 68"
1,2,"Jackson, Kristie",undergraduate,"exam2 (2017), exam2 (2017), exam8 (2018), exam...","66, 69, 85, 62, 92, 72, 44, 50, 23"
2,3,"Curtis, George",graduate,"exam7 (2017), exam6 (2017), exam2 (2017), exam...","68, 73, 62, 72"
3,4,"Casey, Christie",undergraduate,"exam10 (2019), exam4 (2019), exam4 (2019), exa...","41, 52, 52, 70"
4,5,"Yoder, Emily",undergraduate,"exam4 (2019), exam4 (2019), exam4 (2019)","52, 45, 61"
...,...,...,...,...,...
95,96,"Dyer, Benjamin",undergraduate,"exam10 (2019), exam4 (2019), exam6 (2017), exa...","47, 53, 64, 57, 71, 79, 77, 56"
96,97,"Delgado, Jason",undergraduate,"exam1 (2016), exam8 (2018)","72, 83"
97,98,"Cortez, Kenneth",undergraduate,"exam3 (2018), exam5 (2020), exam6 (2017)","83, 55, 71"
98,99,"Short, Elizabeth",undergraduate,"exam3 (2018), exam9 (2018), exam3 (2018), exam...","87, 34, 84, 75, 80, 60, 48, 55, 75"


In [4]:
def normalize_database(non_normalized_db_filename):
#     Normalize 'non_normalized.db'
#     Call the normalized database 'normalized.db'
#     Function Output: No outputs
#     Requirements:
#     Create four tables
#     Degrees table has one column:
#         [Degree] column is the primary key
    
#     Exams table has two columns:
#         [Exam] column is the primary key column
#         [Year] column stores the exam year
    
#     Students table has four columns:
#         [StudentID] primary key column 
#         [First_Name] stores first name
#         [Last_Name] stores last name
#         [Degree] foreign key to Degrees table
        
#     StudentExamScores table has four columns:
#         [PK] primary key column,
#         [StudentID] foreign key to Students table,
#         [Exam] foreign key to Exams table ,
#         [Score] exam score


    # YOUR CODE HERE
    conn = create_connection(non_normalized_db_filename)
    
    sql_statement_exams="Select Exams from Students;"
    df_exams = pd.read_sql_query(sql_statement_exams, conn)
    sql_statement_degrees="Select Degree from Students;"
    df_degrees = pd.read_sql_query(sql_statement_degrees, conn)
    sql_statement_students="Select StudentID,Name,Degree from Students;"
    df_students = pd.read_sql_query(sql_statement_students, conn)
    sql_statement_studentexamscores="Select StudentID,Exams,Scores from Students;"
    df_studentexamscores = pd.read_sql_query(sql_statement_studentexamscores, conn)
    conn.close()
    
    #Exams table
    exams=[]
    for rowid,values in df_exams.iterrows():
        for exam in values[0].split(","):
            exam_name,year=exam.strip().split()
            year=year.replace('(','').replace(')','')
            my_tuple=(exam_name,int(year))
            if(my_tuple not in exams):
                exams.append(my_tuple)
                
    #Degree Table            
    degree=[]
    for rowid,values in df_degrees.iterrows():
        if(values[0] not in degree):
            degree.append(values[0])
    
    
    #Students table
    students=[]
    for rowid,values in df_students.iterrows():
        lastname,firstname=values[1].split(",")
        my_tuple=(values[0],firstname,lastname,values[2])
        #print(my_tuple)
        if(my_tuple not in students):
            students.append(my_tuple)
        
        
            
    # Creating tables 
    conn1 = create_connection('normalized.db')
    create_table_degree_sql=""" CREATE TABLE [Degrees] (
            [Degree] TEXT  NOT NULL PRIMARY KEY); """
            
    create_table(conn1, create_table_degree_sql)
    
    create_table_exams_sql = """ CREATE TABLE [Exams] (
            [Exam] TEXT  NOT NULL PRIMARY KEY,
            [Year] INTEGER); """
    create_table(conn1, create_table_exams_sql)
    
    create_table_students_sql = """ CREATE TABLE [Students] (
            [StudentID] TEXT  NOT NULL PRIMARY KEY,
            [First_Name] TEXT,
            [Last_Name] TEXT,
            [Degree] TEXT,
            FOREIGN KEY(Degree) REFERENCES Degrees(Degree)); """
    create_table(conn1, create_table_students_sql)
    
    create_table_stu_sql = """ CREATE TABLE [StudentExamScores] (
            [PK] INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
            [StudentID] TEXT  NOT NULL,
            [Exam] TEXT,
            [Score] INTEGER,
            FOREIGN KEY(Exam) REFERENCES Exams(Exam)
            FOREIGN KEY(StudentID) REFERENCES Students(StudentID)); """
    create_table(conn1, create_table_stu_sql)
    
    #Inserting data
    
    sql_query_degrees = '''INSERT or replace INTO Degrees(Degree) VALUES(?)'''
    
    cur = conn1.cursor()
    for deg in degree:
        cur.execute(sql_query_degrees,(deg,))
        conn1.commit()
        #cur.close()

    sql_query_exams = '''INSERT or replace INTO Exams VALUES(?,?)'''
    cur = conn1.cursor()
    for exam in exams:
        cur.execute(sql_query_exams,(exam[0],exam[1]))
        conn1.commit()
        
    sql_query_sudents = '''INSERT or replace INTO Students VALUES(?,?,?,?)'''
    cur = conn1.cursor()
    for student in students:
        cur.execute(sql_query_sudents,(student[0],student[1],student[2],student[3]))
        conn1.commit()
    
    sql_query_sudents_scores = '''INSERT or replace INTO StudentExamScores(StudentID,Exam,Score) VALUES(?,?,?)'''
    cur = conn1.cursor()
    rows =[]
    for rowid,values in df_studentexamscores.iterrows():
        score_list=[]
        exam_list=[]
        studentid=[]
        
        #print(values)
    sql_query_sudents_scores = '''INSERT or replace INTO StudentExamScores(StudentID,Exam,Score) VALUES(?,?,?)'''
    cur = conn1.cursor()
    for rowid,values in df_studentexamscores.iterrows():
        score_list=[]
        exam_list=[]
        studentid=[]
        rows =()
            #print(values)
        for exam in values[1].split(","):
            exam_list.append(exam.strip().split()[0])
            studentid.append(values[0])
        rows = tuple(zip(studentid,exam_list,values[2].split(",")))
            #print(rows)
        for row in rows:
            cur.execute(sql_query_sudents_scores,row)
            conn1.commit()
    
   
    conn.close()        
    
    conn1.close()
    
    #Degrees table
    
    

normalize_database('non_normalized.db')    
    

table [Degrees] already exists
table [Exams] already exists
table [Students] already exists
table [StudentExamScores] already exists


In [5]:
normalize_database('non_normalized.db')
conn = create_connection('normalized.db')


table [Degrees] already exists
table [Exams] already exists
table [Students] already exists
table [StudentExamScores] already exists


In [6]:
def ex1():
    # Write an SQL statement that SELECTs all rows from the `Exams` table and sort the exams by Year
    # output columns: exam, year
    
    # YOUR CODE HERE
    
    sql_statement='''select * from Exams ORDER by Year ASC,Exam ASC;'''
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement



 

In [7]:
sql_statement = ex1()
data = pd.read_csv("ex1.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year
0,exam1,2016
1,exam2,2017
2,exam6,2017
3,exam7,2017
4,exam3,2018
5,exam8,2018
6,exam9,2018
7,exam10,2019
8,exam4,2019
9,exam5,2020


In [8]:
def ex2():
    # Write an SQL statement that SELECTs all rows from the `Degrees` table and sort the degrees by name
    # output columns: degree
    
    # YOUR CODE HERE
    sql_statement='''select * from Degrees ORDER by Degree ASC;'''
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [9]:
sql_statement = ex2()
data = pd.read_csv("ex2.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree
0,graduate
1,undergraduate


In [10]:
def ex3():
    # Write an SQL statement that counts the numbers of gradate and undergraduate students
    # output columns: degree, count_degree
    
    # YOUR CODE HERE
    sql_statement='''select Degree,count(StudentID) as count_degree from Students GROUP by Degree;'''
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [11]:
sql_statement = ex3()
data = pd.read_csv("ex3.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Degree,count_degree
0,graduate,35
1,undergraduate,65


In [12]:
def ex4():
    # Write an SQL statement that calculates the exam averages for exams; sort by average in descending order.
    # output columns: exam, year, average
    # round to two decimal places
    
    
    # YOUR CODE HERE
    sql_statement='''SELECT Exams.Exam,Year,round(avg(Score),2) as average from StudentExamScores INNER JOIN Exams on exams.Exam=StudentExamScores.Exam group by Exams.Exam,Year ORDER BY round(avg(Score),2) DESC; '''
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [13]:
sql_statement = ex4()
data = pd.read_csv("ex4.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

,Exam,Year,average
0,exam3,2018,86.16
1,exam8,2018,80.18
2,exam1,2016,75.09
3,exam7,2017,68.96
4,exam2,2017,64.28
5,exam6,2017,61.80
6,exam4,2019,54.03
7,exam5,2020,45.56
8,exam10,2019,44.68
9,exam9,2018,39.43


AssertionError: 

In [ ]:
def ex5():
    # Write an SQL statement that calculates the exam averages for degrees; sort by average in descending order.
    # output columns: degree, average 
    # round to two decimal places
    
    # YOUR CODE HERE
    sql_statement='''select Degree,round(avg(score),2) as average from StudentExamScores se join Students s on s.StudentID=se.StudentID group by degree;'''
    df = pd.read_sql_query(sql_statement, conn)
    display(df)
    return sql_statement

In [ ]:
sql_statement = ex5()
data = pd.read_csv("ex5.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

In [ ]:
def ex6():
    # Write an SQL statement that calculates the exam averages for students; sort by average in descending order. Show only top 10 students
    # output columns: first_name, last_name, degree, average
    # round to two decimal places
    
    # YOUR CODE HERE
    sql_statement='''select First_Name,Last_Name,Degree,round(avg(score)) as average from StudentExamScores se join Students s on s.StudentID=se.StudentID group by se.StudentID order by round(avg(score)) desc,first_name desc LIMIT 10;'''
    df = pd.read_sql_query(sql_statement, conn)
    print(df.round(2))
    return sql_statement
ex6()

In [ ]:
sql_statement = ex6()
data = pd.read_csv("ex6.csv")
cur = conn.cursor()
df = pd.read_sql_query(sql_statement, conn)
assert df.equals(data) == True

In [ ]:
from pandas import DataFrame

Sample = {'Value': [5.00,6.572935,7.21,8.755,9.9989]}
df = DataFrame(Sample, columns= ['Value'])

roundThree = df['Value'].round(decimals=3)
print(roundThree)